In [95]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

In [5]:
# 감자/고구마칩
korea_url = "https://www.coupang.com/np/categories/195270"

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36', "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"}
res = requests.get(korea_url, headers=headers)
res.raise_for_status()
soup = BeautifulSoup(res.text, "lxml")
snack_list =  soup.select(
    '#productList > li'
)
print(snack_list)

[<li class="baby-product renew-badge" data-is-rocket="true" data-product-id="69985840" data-ranking="" data-vendor-item-id="85079702772" id="69985840">
<a class="baby-product-link" data-is-ccid-eligible="false" data-is-rocket="true" data-item-id="17917047191" data-product-id="69985840" data-sns-discount-rate="-1" data-vendor-item-id="85079702772" data-wow-only-instant-discount-rate="-1" href="/vp/products/69985840?itemId=17917047191&amp;vendorItemId=85079702772&amp;sourceType=CATEGORY&amp;categoryId=195170">
<dl class="baby-product-wrap" data-use-data="N">
<dt class="image">

</dt>
<dd class="descriptions">
<div class="badges">
</div>
<div class="name">
    오리온 무뚝뚝감자칩 통후추 소금맛, 10개, 60g
</div>
<div class="price-area">
<div class="price-wrap">
<div class="price">
<em class="sale">
<strong class="price-value">14,350</strong>원
                    
                

                
                    <span class="badge rocket">
<img alt="로켓배송" height="16" src="//image6.coupangcdn.com/imag

In [148]:
# 과자 종류 별 맛 리스트 탐색
def find_flavor_list(category_num):
    url = f"https://www.coupang.com/np/categories/{category_num}?filter=1%23attr_12594%2420402"
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36', "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"}
    html = requests.get(url, headers=headers).text
    soup = BeautifulSoup(html, "lxml") 
    selectors = '#searchOptionForm > div > div > div.newcx-nav > div > div > div.search-filter-options.search-attr_12593-filter > div > ul >li > label'

    flavor_name = soup.select(selectors)
    flavor_list = []
    flavor_num_list = []
    for i in range(len(flavor_name)-1):
        flavor = flavor_name[i].attrs['title']
        flavor_num = flavor_name[i].attrs['for'][4:]    
        flavor_list.append(flavor)
        flavor_num_list.append(flavor_num)
    # test = soup.select(selectors[1])
    # print(test)
    return flavor_list, flavor_num_list

In [149]:
# 195272: 감자과자
flavor_list, flavor_num_list = find_flavor_list(195272)
print(flavor_list)
print(flavor_num_list)

['달콤한맛', '짭짤한맛', '고소한맛', '매콤한맛', '새콤한맛']
['20391', '20392', '20393', '20394', '21103']


In [125]:


def find_snack_link(category_num, start, end):
    if not isinstance(start, int) or not isinstance(start, int):
        raise Exception('Start, End 페이지가 정수 형태가 아닙니다.')

    elif start < 1:
        raise Exception('Start 페이지가 1보다 작습니다')
    
    img_links = []
    name_list = []
    selected_flavor_list = []
    category_list = []
    flavor_list, flavor_num_list = find_flavor_list(category_num)
    print(flavor_list, flavor_num_list)
    for i in range(len(flavor_num_list)):
        for j in range(start, end+1):
            url = f"https://www.coupang.com/np/categories/{category_num}?filter=1%23attr_12594%2420402attr_12593%24{flavor_num_list[i]}?page={j}"
            # url = f"https://www.coupang.com/np/categories/?page={i}"
            headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36', "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"}

            html = requests.get(url, headers=headers).text
            # html.raise_for_status()
            soup = BeautifulSoup(html, "lxml")
            selectors = ['#productList > li > a > dl > dt > img', '#productList > li > a > dl > dd > div.name', '#searchOptionForm > div > div > div.newcx-main > div > div.newcx-main-category-header > h3', '#searchOptionForm > div > div > div.newcx-main > div'] 
            img = soup.select(selectors[0])
            
            name = soup.select(selectors[1])
            category = soup.select(selectors[2])
            # flavor = flavor_list[i]
            # flavor = soup.select(selectors[3])
            # print(category, flavor)
            # print(flavor)
            for k in range(len(img)):
                img_links.append(f"https:{img[k].attrs['src']}")
                name_list.append(name[k].get_text().strip())
                category_list.append(category[0].get_text().strip())
                selected_flavor_list.append(flavor_list[i])        
        
            if j % 10 == 0:
                time.sleep(0.5)
    
    return name_list, img_links, selected_flavor_list, category_list

In [113]:
# 195272: 감자과자
# 195278: 팝콘
name_list, img_links, selected_flavor_list, category_list = find_snack_link(195272, 1, 1)
# print(len(find_snack_link(195272, 1, 1)))

In [154]:
df = pd.DataFrame({'이름': name_list, '이미지' : img_links, '맛': selected_flavor_list, '종류' : category_list})
# print(len(df))
df.to_csv('snack_db.csv', index=False, encoding="utf-8-sig")

140


In [161]:
# 195272: 감자과자
# 195278: 팝콘
# 195279: 나초
# 195284: 비스킷
total_category_list = [195272, 195278, 195279, 195284]
for i in range(len(total_category_list)):
    if i == 0:
        name_list, img_links, selected_flavor_list, category_list = find_snack_link(total_category_list[i], 1, 1)
        df = pd.DataFrame({'이름': name_list, '이미지' : img_links, '맛': selected_flavor_list, '종류' : category_list})
    else:
        name_list, img_links, selected_flavor_list, category_list = find_snack_link(total_category_list[i], 1, 1)
        df_to_insert = pd.DataFrame({'이름': name_list, '이미지' : img_links, '맛': selected_flavor_list, '종류' : category_list})
        df = pd.concat([df, df_to_insert])
        # df = pd.concat(df, df_to_insert, ignore_index=True)


# name_list, img_links, selected_flavor_list, category_list = find_snack_link(195278, 1, 1)
# name_list, img_links, selected_flavor_list, category_list = find_snack_link(195278, 1, 1)



['달콤한맛', '짭짤한맛', '고소한맛', '매콤한맛', '새콤한맛'] ['20391', '20392', '20393', '20394', '21103']
['달콤한맛', '짭짤한맛', '고소한맛', '매콤한맛', '새콤한맛'] ['20391', '20392', '20393', '20394', '21103']
['달콤한맛', '짭짤한맛', '고소한맛', '매콤한맛', '새콤한맛'] ['20391', '20392', '20393', '20394', '21103']
['달콤한맛', '짭짤한맛', '고소한맛', '매콤한맛', '새콤한맛'] ['20391', '20392', '20393', '20394', '21103']


In [163]:
df.to_csv('snack_db.csv', index=True, encoding="utf-8-sig")

In [165]:
data = pd.read_csv("snack_db.csv", sep=",")

In [169]:
data

,이름,이미지,맛,종류
0,"켈로그 프링글스 스윗마요치즈, 110g, 12개입",https://thumbnail6.coupangcdn.com/thumbnails/r...,달콤한맛,감자과자
1,"아로마트러플 블랙써머 트러플 포테이토칩 오리지널, 100g, 1개",https://thumbnail7.coupangcdn.com/thumbnails/r...,달콤한맛,감자과자
2,"가루비 자가비 크림치즈 맛 35g 12봉지 스낵 과자, 기본",https://thumbnail6.coupangcdn.com/thumbnails/r...,달콤한맛,감자과자
3,No Brand 노브랜드 감자칩 고구마칩 110g 3종 오리지널 샤워크림앤어니언 자...,https://thumbnail10.coupangcdn.com/thumbnails/...,달콤한맛,감자과자
4,"켈로그 프링글스 버터카라멜, 110g, 12개입",https://thumbnail7.coupangcdn.com/thumbnails/r...,달콤한맛,감자과자
...,...,...,...,...
724,"롯데제과 롯샌 화이트크림깜뜨, 315g, 2개",https://thumbnail10.coupangcdn.com/thumbnails/...,새콤한맛,비스킷
725,"오리온 마리 비스킷 12p, 222g, 1개",https://thumbnail10.coupangcdn.com/thumbnails/...,새콤한맛,비스킷
726,"롯데샌드 화이트 크림 깜뜨 벌크 6p, 315g, 2개",https://thumbnail10.coupangcdn.com/thumbnails/...,새콤한맛,비스킷
727,"롯데샌드 화이트 크림 깜뜨 벌크 6p, 315g, 4개",https://thumbnail9.coupangcdn.com/thumbnails/r...,새콤한맛,비스킷


In [153]:
df2 = pd.DataFrame({'이름': name_list, '이미지' : img_links, '맛': selected_flavor_list, '종류' : category_list})
new_df = pd.concat([df, df2], ignore_index = True)
new_df.to_csv('snack_db2.csv', index=False, encoding="utf-8-sig")

In [ ]:
# 감자/고구마칩
def find_info(link):
    korea_url = "https://www.coupang.com/np/categories/195270"

    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36', "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"}
    res = requests.get(korea_url, headers=headers)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml")
    snack_list =  soup.select(
        '#productList > li'
    )
    print(snack_list)